In [ ]:
import tweepy
import re
import sys
from datetime import datetime
TWEET_TOPICS = ['pizza']
KAFKA_BROKER = 'localhost:9092'
KAFKA_TOPIC = 'tweets'
class IDPrinter(tweepy.Stream):
    def on_connection_error(self):
        self.disconnect()
    def on_status(self, status):
        tweet = status.text 

        tweet = re.sub(r'RT\s@\w*:\s', '', tweet)
        tweet = re.sub(r'https?.*', '', tweet)
        global producer
        producer.send(KAFKA_TOPIC, bytes(tweet, encoding='utf-8'))

        d = datetime.now()
        print(f'[{d.hour}:{d.minute}.{d.second}] sending tweet')
        print(tweet)


printer = IDPrinter('c9jlD65AuQ21TXr5bpcShbM7z', 'wErzoCpyd8a6lzDngyiNVAgAVI9mlVj2AbzxLU97R6MZORTnBp',
                    '1447786381841862656-bGJzug9J6WeV9osUM20Vkp1i9RExl1', 'ssWWzBnCeHw6bjfMdXSgHeMFflE28o6faYfTjJZv22uf6')
try:
    producer = KafkaProducer(bootstrap_servers=KAFKA_BROKER)
except Exception as e:
    print(f'Error Connecting to Kafka --> {e}')
    sys.exit(1)

printer.filter(track=TWEET_TOPICS)